# Knowledge Distillation

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
tf.random.set_seed(3)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [6]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

student_scratch = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student_scratch",
)

# Clone student for later comparison
#student_scratch = keras.models.clone_model(student)

In [7]:
teacher.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 14, 14, 256)       2560      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 256)       0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 7, 7, 512)         1180160   
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                250890    
                                                           

In [8]:
student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 14, 14, 8)         80        
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 8)         0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 14, 14, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 8)           584       
                                                                 
 flatten_4 (Flatten)         (None, 392)               0         
                                                                 
 dense_4 (Dense)             (None, 10)                3930      
                                                           

In [9]:
student_scratch.summary()

Model: "student_scratch"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 14, 14, 8)         80        
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 14, 14, 8)         0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 1568)              0         
                                                                 
 dense_5 (Dense)             (None, 10)                15690     
                                                                 
Total params: 15770 (61.60 KB)
Trainable params: 15770 (61.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Prepare the train and test dataset.
batch_size = 128
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


11490434/11490434 [==============================] - 1s 0us/step


In [11]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
print("---Testing Accuracy---")
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 17s 5ms/step - loss: 0.1427 - sparse_categorical_accuracy: 0.9571
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0911 - sparse_categorical_accuracy: 0.9733
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0826 - sparse_categorical_accuracy: 0.9759
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0742 - sparse_categorical_accuracy: 0.9788
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0689 - sparse_categorical_accuracy: 0.9806
---Testing Accuracy---
313/313 [==============================] - 1s 3ms/step - loss: 0.1092 - sparse_categorical_accuracy: 0.9730


[0.1092146635055542, 0.9729999899864197]

In [12]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)
print("---Testing Accuracy---")
# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - sparse_categorical_accuracy: 0.8979 - student_loss: 0.3364 - distillation_loss: 0.0153
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - sparse_categorical_accuracy: 0.9539 - student_loss: 0.1616 - distillation_loss: 0.0080
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - sparse_categorical_accuracy: 0.9591 - student_loss: 0.1422 - distillation_loss: 0.0067
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - sparse_categorical_accuracy: 0.9635 - student_loss: 0.1274 - distillation_loss: 0.0061
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - sparse_categorical_accuracy: 0.9670 - student_loss: 0.1159 - distillation_loss: 0.0056
---Testing Accuracy---
313/313 [==============================] - 1s 2ms/step - sparse_categorical_accuracy: 0.9688 - student_loss: 0.1071


[0.9688000082969666, 0.0012940376764163375]

In [13]:
# Train student as done usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2800 - sparse_categorical_accuracy: 0.9208
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1132 - sparse_categorical_accuracy: 0.9671
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0879 - sparse_categorical_accuracy: 0.9739
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0755 - sparse_categorical_accuracy: 0.9776
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0763 - sparse_categorical_accuracy: 0.9760


[0.07634099572896957, 0.9760000109672546]